### Teste -Web Scraping e Crawlers

In [4]:
import requests
from bs4 import BeautifulSoup
import time
import json


In [5]:
url = "http://www.freeproxylists.net"
requisicao = requests.get(url)
site = BeautifulSoup(requisicao.text, "html.parser") #le o html 
print(site.prettify())

<html>
 <head>
  <title>
   Free Proxy Lists
  </title>
  <style type="text/css">
   <!--
div#contents {
width: 350px;
height: 200px;
position: absolute;
top: 50%;
left: 50%;
margin: -100px 0 0 -175px;
padding: 0px;
}
-->
  </style>
  <script async="" defer="" src="https://www.google.com/recaptcha/api.js">
  </script>
 </head>
 <body>
  <div id="contents">
   <center>
    <form action="/" method="post">
     <div class="g-recaptcha" data-sitekey="6LdOqaobAAAAACFOBruA5lJuP0FEKN4vmdz4BMRV">
     </div>
     <br/>
     <input type="submit" value="Submit"/>
    </form>
   </center>
  </div>
 </body>
</html>



In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor


In [7]:
#função para extrair da tabela os elementos (células)
def extrair_dados(linha):
    # Obtenha os elementos da linha (td)
    elementos = linha.find_all("td")
    
    if len(elementos) < 9:
        return None  #Ignore linhas com elementos insuficientes e passe para a próxima
    
    try:
        ip = elementos[0].find("a").text.strip()
        porta = elementos[1].text.strip()
        protocolo = elementos[2].text.strip()
        país = elementos[4].text.strip()
        uptime = elementos[7].text.strip()
    except Exception as e:
        print(f"Erro ao extrair dados da linha: {e}")
        return None
    
    proxy = {
        "ip": ip,
        "porta": porta,
        "protocolo": protocolo,
        "pais": país,
        "uptime": uptime
    }
    
    return proxy


In [11]:
import pandas as pd
import asyncio 
import aiohttp


### Uso de uma lista de ips, para fazer rotação
O resultado não foi o esperado, acredito que pelos ips não serem apropriados.
Esses ips foram adquiridos no site https://hidemyna.me/ e do próprio site  http://www.freeproxylists.net 

In [14]:
proxies= pd.read_excel("proxies_list.xlsx")
proxies

,http://34.75.202.63:80
0,http://103.123.25.65:80
1,http://61.158.175.38:9002
2,http://139.196.214.238:2087
3,http://61.158.175.38:9002
4,http://50.221.203.217:80
5,http://159.203.61.169:8080
6,http://139.59.1.14:3128
7,http://138.68.60.8:8080
8,http://117.251.103.186:8080
9,http://50.206.25.110:80


In [16]:

url = 'http://www.freeproxylists.net'
time_espera = 10

async def checa_proxy(url, proxy):
    try:
        session_timeout = aiohttp.ClientTimeout(
            total=None,
            sock_connect=time_espera,
            sock_read=time_espera
        )
        async with aiohttp.ClientSession(timeout=session_timeout) as session:
            async with session.get(url, proxy=proxy, timeout=time_espera) as resp:
                print(await resp.text())
    except Exception as error:
        print('Proxy com erro', error)

#         #Os proxies lidos de um arquivo Excel são iterados e para cada proxy, 
#         uma tarefa assíncrona é criada usando asyncio.create_task(checa_proxy(url, arquivo_linha)) 
#         e adicionada à lista de tarefas.
#         Usando await asyncio.gather(*tasks), todas as tarefas são executadas em paralelo
async def main():
    tasks = []
    proxies = pd.read_excel("proxies_list.xlsx")
    for arquivo_linha in proxies.iloc[:, 0]:  
        task = asyncio.create_task(checa_proxy(url, arquivo_linha))
        tasks.append(task)
    await asyncio.gather(*tasks)
#Verifica-se se o loop de eventos assíncronos já está em execução. 

if asyncio.get_event_loop().is_running():
    loop = asyncio.get_event_loop()
    loop.create_task(main())
else:
    asyncio.run(main())


I'm busy right now, sorry.
<html><head><title>403 Forbidden</title></head><body><h1>403 Forbidden</h1>Access denied. Your IP address is blacklisted.</body></html>
<html><head><title>403 Forbidden</title></head><body><h1>403 Forbidden</h1>Access denied. Your IP address is blacklisted.</body></html>
<html><head><title>403 Forbidden</title></head><body><h1>403 Forbidden</h1>Access denied. Your IP address is blacklisted.</body></html>
Proxy com erro 
Proxy com erro 
Proxy com erro 
Proxy com erro 
Proxy com erro 
Proxy com erro 
Proxy com erro 
Proxy com erro 
Proxy com erro 
Proxy com erro 
Proxy com erro 
